## Car Pooling Algorithm




In [15]:
from pandas import reset_option
import random
import requests

### Graph

In [16]:
class node:
    def __init__(self, id, type, flag, lat, long):
        self.id = id
        self.type = type # 0: car, 1: start, 2:end
        self.isAccessible = flag
        self.lat = lat
        self.long = long

In [17]:
class edge:
    def __init__(self, id, startNodeId, endNodeId, weight):
        self.id = id
        self.startNodeId = startNodeId
        self.endNodeId = endNodeId
        self.weight = weight

In [18]:
class graph:
    def __init__(self):
        self.adjList = [[]]
        self.nodes = []
        self.nodeCounter = 0
        self.edgeCounter = 0
    
    def getNodeFromId(self, nodeId):
        for n in self.nodes:
            if n.id == nodeId:
                return n
    
    def getEdgeFromEdgeNodes(self, node1, node2):
        for e in self.adjList[node1]:
            if e.endNodeId == node2:
                return e
        return -1
    
    def addNode(self, type, flag, lat, long):
        self.nodes.append(node(self.nodeCounter, type, flag, lat, long))
        self.nodeCounter += 1
        self.adjList.append([])
    
    def addEdge(self, startNode, endNode, weight):
        self.adjList[startNode].append(edge(self.edgeCounter, startNode, endNode, weight))
        self.edgeCounter += 1
    
    def calcEdgeWeight(self, startNode, endNode):
        s = self.getNodeFromId(startNode)
        e = self.getNodeFromId(endNode)

        r = requests.get('http://127.0.0.1:5000/route/v1/driving/'+ str(s.long) +','+ str(s.lat) +';'+ str(e.long) +','+ str(e.lat) +'?steps=false')
        data = r.json()

        return data['routes'][0]['distance']
    
    def addCar(self, loc):
        self.addNode(0, False, loc[1], loc[0])

        for node in self.nodes:
            if node.type == 1:
                self.addEdge(self.nodeCounter-1, node.id, self.calcEdgeWeight(self.nodeCounter-1, node.id))

    def addTrip(self, start, end):
        self.addNode(1, True, start[1], start[0]) # Check the correct order for lat and long with OSRM
        self.addNode(2, False, end[1], end[0])

        for node in self.nodes:
            if node.id != self.nodeCounter-1 and node.id != self.nodeCounter-2: # add an edge from every node to the new start and end nodes 
                self.addEdge(node.id, self.nodeCounter-2, self.calcEdgeWeight(node.id, self.nodeCounter-2))
                if node.type != 0:
                    self.addEdge(node.id, self.nodeCounter-1, self.calcEdgeWeight(node.id, self.nodeCounter-1))

            if node.type != 0 and node.id != self.nodeCounter-2: # add an edge from the new start node to all the other start and end nodes
                self.addEdge(self.nodeCounter-2, node.id, self.calcEdgeWeight(self.nodeCounter-2, node.id))

            if node.type != 0 and node.id != self.nodeCounter-1 and node.id != self.nodeCounter-2: # add an edge from the new end node to all the other end and start nodes
                self.addEdge(self.nodeCounter-1, node.id, self.calcEdgeWeight(self.nodeCounter-1, node.id))
    
    # Functions for testing 

    def printAdjList(self):
        for x in range(0, len(self.adjList)):
            print(x, ": ", end =" ")
            for y in self.adjList[x]:
                print(y.endNodeId, end =" ")
            print("")

### OSRM

In [19]:
pickupNetwork = graph()
# enter coords as long, lat
pickupNetwork.addCar([14.513809277460041, 35.89897453256716]) # valletta

pickupNetwork.addTrip([14.423235598020154, 35.91419450996914], [14.407218690503381, 35.888194056331706]) # Mosta to imdina
pickupNetwork.addTrip([14.49291350433241, 35.87369410066685], [14.513809277460041, 35.89897453256716]) # Marsa to Valletta
pickupNetwork.addTrip([14.349747452527506, 35.952589620545496], [14.488425821564382, 35.88613649037252])  # Mellieha to Hamrun


### Tabu Search

In [20]:
def fitnessFunc(solution, graph):
    value = 0
    for n in range(1, len(solution)):
        n1 = solution[n]
        n2 = solution[n-1]
        value += graph.getEdgeFromEdgeNodes(n2, n1).weight
    
    return value

creating a random solution 

In [21]:
def initSolution(graph):
    solution = []
    visited = [False] * len(graph.nodes)

    for n in graph.nodes:
        if n.type == 0:
            solution.append(n.id)
            visited[n.id] = True
    
    while(visited.count(visited[0]) != len(visited)):
        avalabile = False
        n = random.choice(graph.adjList[solution[-1]]).endNodeId
        if graph.getNodeFromId(n).type == 2:
            start = n -1
            for x in solution:
                if x == start:
                    avalabile = True
                    break
        else:
            avalabile = True

        if visited[n] == False and avalabile:
            visited[n] = True
            solution.append(n)

    return solution
    

neighborhood function

In [22]:
def neighborhood(solution, graph):
    neighborhood = []

    for x in range(1, len(solution)):
        for y in range(x + 1, len(solution)):
            newSolution = solution.copy()
            newSolution[x], newSolution[y] = newSolution[y], newSolution[x]
            if checkSolution(newSolution, graph) == True:
                neighborhood.append(newSolution)
    
    return neighborhood


In [43]:
def checkSolution(solution, graph):
    correct = True
    for n in range(1, len(solution)):
        n1 = solution[n]
        n2 = solution[n-1]
        print(n1, n2)
        edge = graph.getEdgeFromEdgeNodes(n2, n1)
        #print(n2, n1, ": ", edge)

        if edge == -1:
            return False

        if graph.getNodeFromId(n1).type == 2:
            start = n1 -1
            for x in solution[:n]:
                if x == start:
                    correct = True
                    break
                else:
                    correct = False

        if not correct:
            return False

    return True


### Tabu Search

In [24]:
def tabuSearch(graph, iterations, tabuSize, s):
    #bestSolution = initSolution(graph)
    bestSolution = s
    solution = bestSolution
    tabuList = list()
    bestCost = fitnessFunc(bestSolution, graph)
    counter = 1

    while counter <= iterations:
        neighbours = neighborhood(solution, graph)
        found = False # indicates we found a set of different nodes that are not in the tabu list 
        currentBestSolutionIndex = 0
        currentBestSolution = neighbours[currentBestSolutionIndex]
        #print(neighbours)

        while not found and currentBestSolutionIndex < len(neighbours) - 1:
            i = 0
            while i < len(currentBestSolution):
                if currentBestSolution[i] != solution[i]:
                    firstNode = currentBestSolution[i]
                    secondNode = solution[i]
                    break
                i = i + 1

            if [firstNode, secondNode] not in tabuList and [secondNode, firstNode] not in tabuList:
                tabuList.append([firstNode, secondNode]) # add the set to the tabuList
                found = True 

                solution = currentBestSolution
                cost = fitnessFunc(currentBestSolution, graph)

                if cost < bestCost:
                    bestCost = cost
                    bestSolution = solution
            
            else:
                currentBestSolutionIndex += 1
                currentBestSolution = neighbours[currentBestSolutionIndex]
        
        if len(tabuList) >= tabuSize:
            tabuList.pop(0) # removes the oldest element in the list
        
        #print(counter)

        counter = counter + 1
    
    return bestSolution, bestCost



In [25]:
s = initSolution(pickupNetwork)

In [26]:
print(s)

[0, 3, 1, 2, 4, 5, 6]


In [27]:
bestSolution, bestCost = tabuSearch(pickupNetwork, 500000, 7, s)

In [28]:
print("Start solution: ", s, " Cost: ", fitnessFunc(s, pickupNetwork))
print("End solution: ", bestSolution, " Cost: ", bestCost)

Start solution:  [0, 3, 1, 2, 4, 5, 6]  Cost:  70807.2
End solution:  [0, 3, 1, 5, 2, 6, 4]  Cost:  51376.2
0   0
3   1
1   1
5   1
2   2
6   2
4   2


### using multiple cars

### Genetic Algorithm 

In [44]:
class geneticAlgorithm:
    def __init__(self, graph, ps):
        self.graph = graph
        self.populationSize = ps
        self.population = []
        self.fitness = []
        self.normFitness = [999] * ps

        self.initPopulation()
        self.normalizeFitness()
    
    def initPopulation(self):
        for x in range(self.populationSize):
            self.population.append(initSolution(self.graph))
            self.fitness.append(fitnessFunc(self.population[x], self.graph))
    
    def normalizeFitness(self):
        total = 0
        for i in range(self.populationSize):
            total += self.fitness[i]

        for i in range(self.populationSize):
            self.normFitness[i] = self.fitness[i]/total
    
    def pickSolution(self):
        index = 0
        r = random.uniform(0, 1)

        while r > 0:
            r = r - self.normFitness[index]
            index += 1
        
        return self.population[index-1]
    
    def swap(self, solution):
        swapped = False
        while not swapped:
            n1 = random.randint(0, len(self.graph.nodes))
            n2 = random.randint(0, len(self.graph.nodes))

            newSolution = solution.copy()
            newSolution[n1], newSolution[n2] = newSolution[n2], newSolution[n1]
            print(newSolution)
            swapped = checkSolution(newSolution, graph)

        return newSolution
    
    def mutateSolution(self, solution, mutationFactor):
        newSolution = self.swap(solution)
        return newSolution
    
    def findBestSolution(self):
        bestSolution = self.population[0]
        bestCost = self.fitness[0]
        for i in range(1, self.populationSize):
            if self.fitness[i] > bestCost:
                bestCost = self.fitness[i]
                bestSolution = self.population[i]
        
        return bestSolution, bestCost
        
    
    def geneticAlgorithm(self, iterations):
        bestSolution, bestCost = self.findBestSolution()
        for iter in range(iterations):
            newPopultion = []
            newFitness = []
            for s in range(self.populationSize):
                newPopultion.append(self.mutateSolution(self.pickSolution(), 1))
                newFitness.append(fitnessFunc(newPopultion[s], self.graph))
            self.population = newPopultion
            self.fitness = newFitness
            self.normalizeFitness()

            popBestSolution, popBestCost = self.findBestSolution()
            if popBestCost < bestCost:
                bestSolution = popBestSolution
                bestCost = popBestCost
        
        return bestSolution, bestCost

In [45]:
g = geneticAlgorithm(pickupNetwork, 10)

s, c = g.geneticAlgorithm(1000)

print("End solution: ", s, " Cost: ", c)

[0, 3, 4, 5, 1, 2, 6]
3 0


TypeError: getEdgeFromEdgeNodes() missing 1 required positional argument: 'node2'

### testing area 

In [ ]:

r = requests.get('http://127.0.0.1:5000/route/v1/driving/14.423235598020154,35.91419450996914;14.407218690503381,35.888194056331706?steps=false')

data = r.json()

print(data)

{'code': 'Ok', 'routes': [{'geometry': 'ioezE__`wAi@jEj@rGwKfMe@fC`ClG`XlLtCtChFfL~UxMjFbAbCwAxOpCn[eG`O]pQxC`Ce@bBrBtMxE', 'legs': [{'steps': [], 'distance': 4153.7, 'duration': 312.7, 'summary': '', 'weight': 312.7}], 'distance': 4153.7, 'duration': 312.7, 'weight_name': 'routability', 'weight': 312.7}], 'waypoints': [{'hint': 'SegAgGHoAIAAAAAAbAAAAAAAAAAfAAAAAAAAAN0ClkIAAAAA7aepQQAAAABsAAAAAAAAAB8AAAAgAAAAABTcADQCJALEFNwA0wEkAgAAfxFEWslf', 'distance': 20.706096, 'name': 'Triq Sir Arthur Borton', 'location': [14.42304, 35.914292]}, {'hint': '7MIAgAzDAIAcAAAAXAAAAO0AAAB3AAAASikZQo_98UJuL51DgQkeQxwAAABcAAAA7QAAAHcAAAAgAAAAY9fbAPWbIwIz1tsAQpwjAgYAHwFEWslf', 'distance': 28.75223, 'name': 'Triq tal-Infetti', 'location': [14.407523, 35.888117]}]}


In [ ]:
print(data['routes'][0]['distance'])
print(data['routes'][0]['geometry'])

4153.7
ioezE__`wAi@jEj@rGwKfMe@fC`ClG`XlLtCtChFfL~UxMjFbAbCwAxOpCn[eG`O]pQxC`Ce@bBrBtMxE
